In [1]:
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn
from scipy.stats import randint as sp_randint
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import numpy as np
from random import choices
from collections import Counter
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier 
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
synthetic = pd.read_csv("synthetic.csv")
synthetic.pop('gender')
synthetic['admit'] = [0 if x == False else 1 for x in synthetic['admit']]
synthetic.head()

,admit,iq,sat
0,0,92.472816,959.728157
1,0,77.055728,862.557282
2,0,96.986655,1046.866552
3,0,104.129996,1105.299959
4,0,101.215587,1047.155871


In [3]:
def add_Fx(df_p,df_n,ppc, npc):
    """
    
    Add sensitive feature {0,1} to the dataframe
    
    parameter:
    - df_p   : dataframe for Y = 1
    - df_n   : dataframe for Y = 0
    - ppc    : percentage of S = 1 in Y = 0
    - npc    : percentage of S = 1 in Y = 0
    
    output:
    - X      : 8-dim array of independent variable
    - y      : 1-dim array of target variable
    
    """
    
    
    df_p.loc[:, 'Feature_X'] = choices([0,1],cum_weights = [ppc,100], k = len(df_p))
    df_n.loc[:, 'Feature_X'] = choices([0,1],cum_weights = [npc,100], k = len(df_n))
    dall = df_p.append(df_n)   
    
#     print("Distribution of Sensitive Attribute in Y = 1: {}".format(Counter(df_p['Feature_X'])))
#     print("Distribution of Sensitive Attribute in Y = 0: {}".format(Counter(df_n['Feature_X'])))
#     print("Distribution of Y (total): {}".format(Counter(dall['Class'])))
#     print("Distribution of Sensitive Attribute in Y (total): {}".format(Counter(dall['Feature_X'])))
    y = dall.pop("admit").values
    X = dall.values
    #print("Dimension of X after adding sensitive attribute: {}".format(X.shape))
    #print("Shape of y: {}".format(y.shape))
    return X, y 

def df_subsample_pos_class(all_df, rpc = 0, subsample = True):
    
    """
    
    Add sensitive feature {0,1} to the dataframe
    
    parameter:
    - all_df : original dataframe
    - rpc    : requested positive percentage to subsample
    
    output:
    - df_p      : dataframe for Y = 1 after subsampling
    - df_n      : dataframe for Y = 0
    
    """
    
    if subsample :
        rp = rpc/100 
        df_p = all_df[all_df['admit']==1].copy()
        df_n = all_df[all_df['admit']==0].copy()
        np = len(df_p)
        nn = len(df_n)
        perc_p = np/(np+nn)
    
        if rp > perc_p:
            print('Requested positive percentage (pcpc) is too high.',perc_p)
            return df_p, df_n
    
        np_dash = rp/(1-rp)* nn
        df_p = df_p.sample(int(np_dash+0.5))
    
        #print(np,nn,'--',np_dash)
        return df_p, df_n
    else:
        df_p = all_df[all_df['admit']==1].copy()
        df_n = all_df[all_df['admit']==0].copy()
        return df_p,df_n
    
def df_count_feat_val_match(df1, f1, v1, f2, v2):
    return len (df1[(df1[f1]==int(v1)) & (df1[f2]==int(v2))])

def optimize_model(clf,param,Xtrain,Xtest,y_train,y_test,scoring,cv):
    grid = RandomizedSearchCV(clf,param,cv=cv,scoring=scoring,n_jobs=-1,n_iter=50)
    grid.fit(Xtrain,y_train)
#     print("Classifier: {}".format(clf.__class__.__name__))
#     print("Best parameter: {}".format(grid.best_params_))
#     print("Best {}: {}".format(str(scoring),grid.best_score_))
#     print("-"*30)
    y_pred = grid.predict(Xtest)
    return y_pred,grid.best_params_

## return best param
## return best score

def underestimation_score(y_true,y_pred,SA):
    """
    
    parameter:
    - y_true : ground truth for prediction outcomes
    - y_pred : predicted outcomes
    - SA     : sensitive attributes
    
    output:
    - us_0: underestimation for S = 0
    - us_1: underestimation for S = 1
    
    """
    mydict = {}
    mydict['actual'] = y_true
    mydict['predicted'] = y_pred
    mydict['sex'] = SA
    us = pd.DataFrame(mydict)

    P_dash_FX0 = df_count_feat_val_match(us, 'predicted', 1, 'sex',0)
    P_FX0 = df_count_feat_val_match(us, 'actual', 1, 'sex',0)
    Bias_FX0 = P_dash_FX0/P_FX0
    
    if P_FX0 == 0:
        print("Divsion by zero detected!")
               
    return Bias_FX0

def evaluate_model(y_true,y_pred,SA):
    """
    
    parameter:
    - y_true : ground truth for prediction outcomes
    - y_pred : predicted labels
    - y_prob : predicted probability
    - SA     : sensitive attributes in the test data
    
    output:
    - accuracy : accuracy scores
    - rocauc   : roc auc scores
    - us_0     : underestimation score for S = 0
    - us_1     : underestimation score for S = 1
    
    """
    Counter
    accuracy = balanced_accuracy_score(y_true,y_pred)
    b = underestimation_score(y_true,y_pred,SA)

    
    ## add counter for SA
    ## add counter for class imbalance
    ## add TP TN etc
    
    return accuracy,b


    
def preprocess_data(X,y):
    """
    Numerical features are scaled using MinMaxScaler, while categorical features one-hot-encoded.
    
    parameter:
    - X    : 8-dim array of independent variable
    - y    : 1-dim array of target variable
    
    output:
    - Xtrain      : 8-dim array containing independent variable in the train test
    - Xtest       : 8-dim array containing independent variable in the test set
    - y_train     : 1-dim array of target variable in the train set
    - y_test      : 1-dim array of target variable in the test set
    
    """
    
    minority_in_Pos = 0
    
    while not minority_in_Pos:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True, stratify = y)
        temp = pd.DataFrame({'Feature_X': X_test[:,2], 'admit': y_test}, columns=['Feature_X', 'admit'])
#         print(df_count_feat_val_match(temp, 'admit', 1, 'Feature_X',0) )
        minority_in_Pos = df_count_feat_val_match(temp, 'admit', 1, 'Feature_X',0)    

    minmax = MinMaxScaler()
    xtrain_num = minmax.fit_transform(X_train[:,0:2])
    xtest_num = minmax.transform(X_test[:,0:2])
    
    Xtrain = np.hstack((xtrain_num,X_train[:,2].reshape(-1,1)))
    Xtest = np.hstack((xtest_num,X_test[:,2].reshape(-1,1)))

#     print("Shape of train data: {}".format(Xtrain.shape))
#     print("Shape of test data : {}".format(Xtest.shape))
    
    return Xtrain,Xtest,y_train,y_test

def US_scorer(clf, X_tst, y_tst):
    y_pred = clf.predict(X_tst)
    Fx_tst = [int(i) for i in X_tst[:,-1]]
    US_s = underestimation_score(y_tst,y_pred,Fx_tst)
    try:
        US = 1/(abs(1-US_s))
    except:
        US= 1/(abs((1-US_s))+0.0001)
    return US

def oversample_minority_smote_adasyn(X_N,Y_N,X_P,Y_P):

    
    from imblearn.over_sampling import SMOTE

    #oversample (SMOTE) minority for both y=0 and y=1
    X_N_smote, Y_N_smote = SMOTE(sampling_strategy='minority',n_jobs=-1,random_state=0).fit_resample(X_N, Y_N)
    X_P_smote, Y_P_smote = SMOTE(sampling_strategy='minority',n_jobs=-1,random_state=0).fit_resample(X_P, Y_P)
    X_N_smote['gender'] = Y_N_smote
    X_N_smote['admit'] = np.zeros(Y_N_smote.shape)
    X_P_smote['gender'] = Y_P_smote
    X_P_smote['admit'] = np.ones(Y_P_smote.shape)
    train_smote = pd.concat([X_N_smote,X_P_smote])

    #get xtrain and ytrain for SMOTE
    y_train_smote = train_smote['admit']
    train_smote.pop('admit')
    Xtrain_smote = train_smote
    
    return Xtrain_smote,y_train_smote

from sklearn.metrics import balanced_accuracy_score

def custom_loss_function(clf,X_tst,y_test):
    y_pred = clf.predict(X_tst)
    Fx_tst = [int(i) for i in X_tst[:,-1]]
    
    US_s = underestimation_score(y_test,y_pred,Fx_tst)
    try:
        US = 1/(abs(1-US_s))
    except:
#         print("smoothing added")
        US= 1/(abs((1-US_s))+0.0001)

    accuracy = balanced_accuracy_score(y_test,y_pred)
    
    weight_ACC = 0.5
    
    return (weight_ACC*accuracy) + ((1-weight_ACC) * US)

In [4]:
models = MLPClassifier(random_state=0,max_iter=200000) 
params = {'hidden_layer_sizes': sp_randint(1, 200),\
          'activation':['tanh','relu','logistic'],'solver':['adam','sgd','lbfgs'],\
                        'alpha':stats.reciprocal(a=1e-7,b=1e3), 'learning_rate': ['constant','adaptive']}

In [5]:
accuracies = []
accuracies_1 = []
accuracies_3 = []
accuracies_4 = []

bis = []
bis_1 = []
bis_3 = []
bis_4 = []

for rep in range(20):
    print("#"*30)
    print(rep+1)
    
    df_p,df_n = df_subsample_pos_class(synthetic,25)
    X,y = add_Fx(df_p,df_n,40,40)
    Xtrain,Xtest,y_train,y_test = preprocess_data(X,y)
    #dataset description
    train = pd.DataFrame(np.hstack((Xtrain,y_train.reshape(-1,1))), columns=['iq','sat','gender','admit'])
    NF = train[(train['gender']==0) & (train['admit']==0)]
    NM = train[(train['gender']==1) & (train['admit']==0)]
    PF = train[(train['gender']==0) & (train['admit']==1)]
    PM = train[(train['gender']==1) & (train['admit']==1)]

    print("Training set: ")
    print("-"*30)
    print("P(Y = 0 | S = 0) samples: {}".format(len(NF)))
    print("P(Y = 0 | S = 1) samples: {}".format(len(NM)))
    print("P(Y = 1 | S = 0) samples: {}".format(len(PF)))
    print("P(Y = 1 | S = 1) samples: {}".format(len(PM)))



    # split dataset by classes
    N = pd.concat([NF.iloc[:,0:3],NM.iloc[:,0:3]])
    Y_N = N['gender']
    N.pop('gender')
    X_N = N
    P = pd.concat([PF.iloc[:,0:3],PM.iloc[:,0:3]])
    Y_P = P['gender']
    P.pop('gender')
    X_P = P


    y_pred,_ = optimize_model(models,params,Xtrain,Xtest,y_train,y_test,'balanced_accuracy',10)

    ## SMOTE & ADASYN
    Xtrain_smote,y_train_smote = oversample_minority_smote_adasyn(X_N,Y_N,X_P,Y_P)
    y_pred_1,_ = optimize_model(models,params,Xtrain_smote,Xtest,y_train_smote,y_test,'balanced_accuracy',10)
    
    to_be_sampled = PM.shape[0]-PF.shape[0]
    ## counterfactual - 1
    sampled_NF = NF.sample(n=to_be_sampled, random_state=1)
    sampled_NF['admit'] = np.ones(to_be_sampled)
    counterfactual_1 = pd.concat([sampled_NF,PF,PM,NF,NM])
    y_train_counterfactual_1 = counterfactual_1['admit']
    counterfactual_1.pop('admit')
    Xtrain_counterfactual_1 = counterfactual_1
    y_pred_3,_ = optimize_model(models,params,Xtrain_counterfactual_1,Xtest,y_train_counterfactual_1,y_test,'balanced_accuracy',10)

    ## counterfactual - 2
    sampled_PM = PM.sample(n=to_be_sampled, random_state=1)
    sampled_PM['admit'] = np.ones(to_be_sampled)
    counterfactual_2 = pd.concat([sampled_PM,PF,PM,NF,NM])
    y_train_counterfactual_2 = counterfactual_2['admit']
    counterfactual_2.pop('admit')
    Xtrain_counterfactual_2 = counterfactual_2
    y_pred_4,_ = optimize_model(models,params,Xtrain_counterfactual_2,Xtest,y_train_counterfactual_2,y_test,'balanced_accuracy',10)


    accuracy,bi = evaluate_model(y_test,y_pred,Xtest[:,2].ravel()) 
    accuracy_1,bi_1 = evaluate_model(y_test,y_pred_1,Xtest[:,2].ravel()) 
    accuracy_3,bi_3 = evaluate_model(y_test,y_pred_3,Xtest[:,2].ravel()) 
    accuracy_4,bi_4 = evaluate_model(y_test,y_pred_4,Xtest[:,2].ravel()) 
    
    accuracies.append(accuracy)
    accuracies_1.append(accuracy_1)
    accuracies_3.append(accuracy_3)
    accuracies_4.append(accuracy_4)
    
    bis.append(bi)
    bis_1.append(bi_1)
    bis_3.append(bi_3)
    bis_4.append(bi_4)
    print("accuracy:")
    print(np.median(accuracies),np.median(accuracies_1),np.median(accuracies_3),np.median(accuracies_4))
    print("underestimation:")
    np.median(bis),np.median(bis_1),np.median(bis_3),np.median(bis_4)

##############################
1
Training set: 
------------------------------
P(Y = 0 | S = 0) samples: 819
P(Y = 0 | S = 1) samples: 1267
P(Y = 1 | S = 0) samples: 274
P(Y = 1 | S = 1) samples: 421
accuracy:
0.6482102908277405 0.6236017897091722 0.6157718120805369 0.6308724832214765
underestimation:
##############################
2
Training set: 
------------------------------
P(Y = 0 | S = 0) samples: 807
P(Y = 0 | S = 1) samples: 1279
P(Y = 1 | S = 0) samples: 272
P(Y = 1 | S = 1) samples: 423
accuracy:
0.6490492170022372 0.6322706935123042 0.6305928411633109 0.6476510067114094
underestimation:
##############################
3
Training set: 
------------------------------
P(Y = 0 | S = 0) samples: 820
P(Y = 0 | S = 1) samples: 1266
P(Y = 1 | S = 0) samples: 272
P(Y = 1 | S = 1) samples: 423
accuracy:
0.6498881431767338 0.6409395973154363 0.645413870246085 0.6644295302013423
underestimation:
##############################
4
Training set: 
------------------------------
P(Y = 0 | S =

exception calling callback for <Future at 0x1847d9225e0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\willi\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\willi\anaconda3\lib\site-packages\joblib\parallel.py", line 366, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\willi\anaconda3\lib\site-packages\joblib\parallel.py", line 799, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\willi\anaconda3\lib\site-packages\joblib\parallel.py", line 866, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\willi\anaconda3\lib\site-packages\joblib\parallel.py", line 784, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\Users\willi\anaconda3\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    future = self._workers.submit(SafeFunction(func))


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [7]:
np.median(accuracies),np.median(accuracies_1),np.median(accuracies_3),np.median(accuracies_4)

(0.6448545861297539,
 0.6445749440715883,
 0.6448545861297539,
 0.6613534675615212)

In [8]:
np.median(bis),np.median(bis_1),np.median(bis_3),np.median(bis_4)

(0.5237327188940092,
 0.5871752783328577,
 0.7892290869327501,
 0.5375576036866359)